In [1]:
from graviz import graph_processing
import json
import pickle
from BuildGraphModule import *
import networkx as nx
import community as com
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Adding some useful statistics to the nodes in the graph
### If you run from scratch and there are no precomputed files run following cells skip loading graph agian in next section, otherwise start in the next section

In [ ]:
graph = graph_processing.load_graph('graph.json', 'JSON')

In [ ]:
# it takes approximately 10 minutes using 8 cores
graph, degree_nodes, eigenvector_nodes, betweenness_nodes = graph_processing.add_centrality_stats(graph)

In [ ]:
graph_processing.save_graph(graph, 'graviz/graph_centrality_serialized.json', 'JSON')

# Trimming graph according to eigenvector centrality measure

In [2]:
graph = graph_processing.load_graph('graviz/graph_centrality_serialized.json', 'JSON')
eigenvector_nodes = nx.get_node_attributes(graph, 'eigenvector_centrality')
betweenness_nodes = nx.get_node_attributes(graph, 'betweenness_centrality')

In [3]:
graph = graph_processing.reduce_graph_size(graph, eigenvector_nodes, threshold=2500)

In [4]:
print(nx.info(graph))

Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 17492
Average degree:  13.9936


In [5]:
with open('paper.json', 'r') as f:
    data = json.load(f)

In [6]:
# Uniform strings - Authors
for i in list(data.keys()):
    try:
        for j in range(len(data[i]['Authors'])):
            t = unidecode(data[i]['Authors'][j])
            t.encode("ascii")
            data[i]['Authors'][j] = t.lower().replace('.','')
    except:
        continue

In [7]:
# Uniform strings - EPFL Authors
for i in list(data.keys()):
    dic = []
    try:
        for j in range(len(data[i]['Epfl authors'])):
            string = list(data[i]['Epfl authors'][j].keys())[0]
            t = unidecode(string)
            t.encode("ascii")
            dic += [{t.lower().replace('.','') : list(data[i]['Epfl authors'][j].values())[0]}]
        data[i]['Epfl authors'] = dic
    except:
        continue

In [8]:
data_cut = {}
for paper in list(data.keys()):
    try:
        if int(data[paper]['Publication date']) >= 1993:
            data_cut[paper] = data[paper]
    except:
        continue

In [9]:
# (lab,website)
dict_lab_site = {}
# (lab, num publications)
dict_labs = defaultdict(int)
# (lab, list of papers)
dict_labs_paper = defaultdict(list)
i = 0
for paper in list(data_cut.keys()):
    try:
        if len(data_cut[paper]['Labs involved']) == 1:
            dict_lab_site[list(data_cut[paper]['Labs involved'][0].keys())[0]] = list(data_cut[paper]['Labs involved'][0].values())[0]
            dict_labs[list(data_cut[paper]['Labs involved'][0].keys())[0]] += 1
            dict_labs_paper[list(data_cut[paper]['Labs involved'][0].keys())[0]] += [paper]
            
        else:
            for lab in data_cut[paper]['Labs involved']:
                dict_lab_site[list(lab.keys())[0]] = list(lab.values())[0]     
                dict_labs[list(lab.keys())[0]] += 1
                dict_labs_paper[list(lab.keys())[0]] += [paper]
    except:
        continue

In [10]:
# Get dictionary (lab,school)
def school_lab(dict_lab_site, name_school):
    school = pickle.load(open(name_school + '.p', 'rb')) # Recall 
    lab_school = {}
    for i in list(dict_lab_site.keys()):
        for nodes in list(school.keys()):
            list_node = school[nodes]
            if i in list_node:
                lab_school[i] = name_school
                break
            else:
                continue
    return lab_school

In [11]:
list_schools = ['ENAC', 'SB', 'IC', 'STI', 'SV']
dict_lab_school = {}
for school in list_schools:
    dict_lab_school = {**dict_lab_school, **school_lab(dict_lab_site, school)}

In [12]:
author_lab = pickle.load(open('author_lab.p', 'rb'))

In [13]:
dict_schools = {'ENAC':1, 'SB':2, 'IC':3, 'STI':4, 'SV':5}

In [14]:
authors = nx.get_node_attributes(graph, 'author_id')

In [15]:
# when scraped fully
#proper_values = [dict_schools[dict_lab_school[author_lab[authors[node]]]] for node in graph.nodes()]

# bypass for now
proper_values = []
for node in graph.nodes():
    try:
        proper_values += [dict_schools[dict_lab_school[author_lab[authors[node]]]]]
    except:
        proper_values += [0]

In [16]:
parts = com.best_partition(graph)

In [17]:
values = [parts.get(node) for node in graph.nodes()]

In [18]:
nodes_positions = nx.spring_layout(graph)

In [19]:
nodes_positions_mod = nodes_positions
max_label = max(values)

In [20]:
# parameters for community shift
#r = 1
a = 0.6
b = 0.85
for key in nodes_positions_mod:
    # circle or elipse placement
    #nodes_positions_mod[key] += [r/2 * np.cos(2.0 * np.pi * parts[key] / max_label), r * np.sin(2.0 * np.pi * parts[key] / max_label)]
    # logarithmic spiral placement
    nodes_positions_mod[key] += [a * np.exp(b * (1 - parts[key] / max_label)) * np.cos(2.0 * np.pi * (1 - parts[key] / max_label)), a * np.exp(b * (1 - parts[key] / max_label)) * np.sin(2.0 * np.pi * (1 - parts[key] / max_label))]

In [21]:
plt.axis("off")
nx.draw_networkx(graph, pos=nodes_positions_mod, cmap=plt.get_cmap("jet"), node_color=values, node_size=50, with_labels=False)

In [22]:
plt.axis("off")
nx.draw_networkx(graph, pos=nodes_positions_mod, cmap=plt.get_cmap("jet"), node_color=proper_values, node_size=50, with_labels=False)